# Extracting model information

In the study of ODE systems, there are many calculations which are commonly performed and PyGOM has some functionality to make these easier.
We will again use the SIR model (REF) as our model system, but this time we will make use of the common_models feature, where many predefined models are stored.
This means we can avoid having to define the model from scratch again.

In [1]:
from pygom import common_models
ode = common_models.SIR_N_stochastic()

## Verification

As seen previously, the `get_ode_eqn` function allows us to check our system of ODEs.

In [ ]:
ode.get_ode_eqn()

```{tip}
In addition to showing the equation form of the ODEs, we can also display them as either symbols or latex code, which can save some extra typing when porting the equations to another document.```

In [ ]:
ode.print_ode()
ode.print_ode(True)

We can also check in terms of transitions

In [ ]:
ode.get_transition_matrix()

And even inspect the transition diagram

In [ ]:
ode.get_transition_graph();

## Algebraic insights

We briefly outline some of the algebraic results which can be quickly accessed by PyGOM.

Firstly, we can check if our system is linear

In [ ]:
ode.linear_ode()

For stability analysis and speeding up numerical integrators, it may be useful to know the Jacobian, Hessian or gradient

In [ ]:
ode.get_jacobian_eqn()

In [ ]:
ode.get_hessian_eqn()

In [ ]:
ode.get_grad_eqn()

## Epidemiology specific insights

The {func}`.R0` function may be used to obtain the basic reproduction number $R_{0}$.
We must specify which state represents the *disease state*, which in this case is the state **I**.
#TODO is this the disease state, or the infectious state?

In [2]:
from pygom.model.epi_analysis import *

R_0=R0(ode, 'I')
print(R_0)

Matrix([[S*beta/N]])
Matrix([[gamma]])
[S*beta/(N*gamma)]
{S: 0, R: 0}
[0]
[0]


We may also wish to get the $R_{0}$ in pure algebraic terms.
This can be achieved by the following few lines of code. Note that the result below is slightly different from the one above.
The difference is due to the internal working of the functions, where {func}`.getR0` computes the disease-free equilibrium value for the states and substitutes them back into the equation.

In [ ]:
F, V = disease_progression_matrices(ode, 'I')

e = R0_from_matrix(F, V)
print(e)

To replicate the output before, we have to find the values where the
disease-free equilibrium will be achieved. Substitution can then be
performed to retrieve $R_{0}$ in pure parameters.

In [ ]:
dfe = DFE(ode, ['I'])

print(dfe)
print(e[0].subs(dfe))